#  Import Libraries

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
from keras import backend as K

# Load MNIST Dataset

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("MNIST training set  len ={}".format(len(X_train)))
print("MNIST training set  len ={}".format(len(X_test)))

11493376/11490434 [==============================] - 6s 1us/step
MNIST training set  len =60000
MNIST training set  len =10000


# Scalling Images

In [3]:
#--------------------------------------
# Scalling Images by 255
#--------------------------------------
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#  Change Output to Categorical

In [4]:
#--------------------------------------
# Change Output to Categorical
#--------------------------------------
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train[:5, :]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

#  MNIST-MLP

##  Build MLP layers

In [6]:
#--------------------------------------
# Build MLP NN mlp
#--------------------------------------
# define the mlp
mlp = Sequential()
mlp.add(Flatten(input_shape=X_train.shape[1:]))
mlp.add(Dense(512, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(512, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy', optimizer='adam', 
    metrics=['accuracy'])

# summarize the mlp
mlp.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
__________________________________________________

##  Trainning MLP

In [7]:
batch_size = 128
epochs = 12
mlp.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/12
469/469 [==============================] - 8s 14ms/step - loss: 0.4524 - accuracy: 0.8644 - val_loss: 0.1026 - val_accuracy: 0.9673
Epoch 2/12
469/469 [==============================] - 6s 12ms/step - loss: 0.1049 - accuracy: 0.9672 - val_loss: 0.0822 - val_accuracy: 0.9740
Epoch 3/12
469/469 [==============================] - 6s 13ms/step - loss: 0.0717 - accuracy: 0.9777 - val_loss: 0.0706 - val_accuracy: 0.9786
Epoch 4/12
469/469 [==============================] - 6s 13ms/step - loss: 0.0516 - accuracy: 0.9835 - val_loss: 0.0779 - val_accuracy: 0.9751
Epoch 5/12
469/469 [==============================] - 6s 13ms/step - loss: 0.0466 - accuracy: 0.9851 - val_loss: 0.0755 - val_accuracy: 0.9767
Epoch 6/12
469/469 [==============================] - 6s 12ms/step - loss: 0.0369 - accuracy: 0.9874 - val_loss: 0.0646 - val_accuracy: 0.9813
Epoch 7/12
469/469 [==============================] - 6s 13ms/step - loss: 0.0293 - accuracy: 0.9901 - val_loss: 0.0652 - val_accuracy: 0.9812

## Model Evaluation

In [8]:
score = mlp.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07863231748342514
Test accuracy: 0.9793999791145325


## Model Saving 

In [9]:
mlp.save('MLP-MNIST.h5')

#  MNIST - CNN

##  Prepare Image for CNN

In [10]:
K.image_data_format()

'channels_last'

In [11]:
# input image dimensions
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

 ## Build CNN

In [12]:
cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(num_classes, activation='softmax'))

cnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

## Training CNN

In [13]:
cnn.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/12
469/469 [==============================] - 118s 247ms/step - loss: 0.4440 - accuracy: 0.8600 - val_loss: 0.0562 - val_accuracy: 0.9816
Epoch 2/12
469/469 [==============================] - 110s 236ms/step - loss: 0.0823 - accuracy: 0.9764 - val_loss: 0.0464 - val_accuracy: 0.9836
Epoch 3/12
469/469 [==============================] - 103s 219ms/step - loss: 0.0620 - accuracy: 0.9821 - val_loss: 0.0383 - val_accuracy: 0.9882
Epoch 4/12
469/469 [==============================] - 95s 202ms/step - loss: 0.0529 - accuracy: 0.9847 - val_loss: 0.0351 - val_accuracy: 0.9896
Epoch 5/12
469/469 [==============================] - 95s 202ms/step - loss: 0.0471 - accuracy: 0.9860 - val_loss: 0.0328 - val_accuracy: 0.9898
Epoch 6/12
469/469 [==============================] - 92s 196ms/step - loss: 0.0475 - accuracy: 0.9861 - val_loss: 0.0282 - val_accuracy: 0.9915
Epoch 7/12
469/469 [==============================] - 92s 196ms/step - loss: 0.0464 - accuracy: 0.9864 - val_loss: 0.0349 - val

## Model Evaluation 

In [14]:
score = cnn.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.033067964017391205
Test accuracy: 0.9908999800682068


In [16]:
X_test[0].shape

(28, 28, 1)

In [19]:
cnn.predict(X_test[0].reshape(1,28,28,1))

array([[6.2048650e-10, 4.3249182e-09, 1.3217575e-06, 1.9632378e-06,
        7.1918578e-09, 1.5210229e-10, 4.0860876e-14, 9.9998891e-01,
        7.1774046e-08, 7.7997938e-06]], dtype=float32)

##  Model Saving

In [ ]:
cnn.save('CNN-MNIST.h5')